#### **🧱 Structured Outputs (JSON) in GenAI**

#### 🧠 Why Structured Outputs (JSON) Matter in GenAI

#### **What is a Structured Output?**
A **structured output** is a response from an LLM that follows a **strict, predictable format**,  
such as JSON, instead of free-form text.

#### **Example**

```json
{
  "name": "Python",
  "type": "programming_language",
  "difficulty": "beginner"
}

This format is **machine-readable**, not just human-readable.

#### **Real-World Scenarios Where JSON Is Mandatory**

#### **1️⃣ APIs**
APIs expect:
- fixed keys
- predictable values
- no extra text

If the model adds:
"Here is your JSON:"
→ your API breaks.

---

#### **2️⃣ Agents & Tools**
Agents rely on:
- exact fields
- correct data types
- consistent structure

One extra sentence can:
- crash the agent
- trigger wrong actions

---

#### **3️⃣ Automation Pipelines**
Examples:
- SQL generation
- Workflow automation
- Data extraction

These systems **parse** outputs.
Parsing free text is unreliable.


#### **What Goes Wrong Without Structured Output**

Common production failures:
- Extra explanations added by the model
- Invalid JSON syntax
- Missing keys
- Wrong data types
- Hallucinated fields

These are NOT model bugs.
They are **design mistakes**.


#### **Interview-Level Insight (Very Important)**

Question:
"Why do we need structured outputs from LLMs?"

Strong answer:

"Because production systems need deterministic, parseable responses.
Free text is unsafe for automation."

Weak answer:
"JSON looks cleaner."


#### **Mental Model to Remember**

LLMs are great at:
- Generating language

They are NOT:
- Databases
- APIs
- State machines

Structured output is how we make LLMs
**behave safely inside software systems**.


#### **Today’s Goal**

By the end of today, I will be able to:
- Explain why JSON output is required
- Identify when free-text breaks systems
- Design safer GenAI applications

**✅ Summary (Rule Applied)**

Structured output = predictable, machine-readable format

JSON is mandatory for production GenAI systems

Free-text causes failures in APIs, agents, automation

This is a design problem, not a model problem

#### **Client Configuration**

In [48]:
# Importing the 'nbimporter' package which allows you to import Jupyter notebooks as Python modules
import nbimporter

# Commented out line (doesn't execute): The '%run' magic command would run the '01_grokai_chat_intro.ipynb' notebook in Jupyter.
## %run 01_grokai_chat_intro.ipynb

# Importing 'sys' to interact with the Python runtime environment (for modifying the system path)
import sys

# Importing 'os' to work with the operating system, like handling file paths
import os

# Add the absolute path to the project directory ('genai_project') to the Python path
# This allows Python to find and import modules from this directory
sys.path.append(os.path.abspath("C:/Users/dhira/Desktop/genai_project"))

# Import the 'client' object from the 'grokai_client_setup.py' file located in the project directory
# This client is likely responsible for setting up communication with the GrokAI system
from grokai_client_setup import client


#### **🎯 Topic 2 Goal — Naive JSON Generation**



In this topic, we will:
- Ask the LLM to return JSON
- Do it in a *naive way* (as most beginners do)
- Observe what breaks and why this approach is unsafe

This is a **controlled failure** to build intuition.

#### **Naive JSON Request (Beginner Approach)**

In [49]:
# ============================================================
# 📘 SECTION 1 — Naive JSON Prompt
# ------------------------------------------------------------
# Why this prompt?
#   - This is how beginners usually ask for JSON
#   - We do NOT enforce strict rules
#   - We want to see what goes wrong
# ============================================================

prompt = """

Return user details in JSON format.
Name: Dhiru
Age:36
Skill: Python, SQL, GenAI

"""
# ============================================================
# 📘 SECTION 2 — Call the LLM (No Enforcement)
# ------------------------------------------------------------
# Why this call is naive:
#   - We only *ask* for JSON
#   - We do NOT restrict extra text
#   - We do NOT validate the output
# ============================================================

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role":"user","content":prompt}],
    temperature=0.4,
    max_tokens=200
)

# ============================================================
# 📘 SECTION 3 — Print Raw Output
# ------------------------------------------------------------
# Why print raw output?
#   - To see exactly what the model returns
#   - To inspect extra text, formatting issues, etc.
# ============================================================

raw_output = response.choices[0].message.content


print("===== RAW MODEL OUTPUT =====\n")
print(raw_output)

===== RAW MODEL OUTPUT =====

Here's a Python code snippet that returns user details in JSON format:

```python
import json

# Define user details
user_details = {
    "name": "Dhiru",
    "age": 36,
    "skills": ["Python", "SQL", "GenAI"]
}

# Convert user details to JSON format
json_data = json.dumps(user_details, indent=4)

# Print JSON data
print(json_data)
```

When you run this code, it will output the following JSON data:

```json
{
    "name": "Dhiru",
    "age": 36,
    "skills": [
        "Python",
        "SQL",
        "GenAI"
    ]
}
```

You can also use Python's built-in `json.dumps()` function with the `sort_keys` parameter to sort the keys in the JSON output:

```python
json_data = json.dumps(user_details, indent=4, sort_keys=True)
```

This will


#### **🔍 What Went Wrong (Naive JSON)**

Even though we asked for JSON:
- The model added extra text
- The output is not guaranteed to be pure JSON
- A parser would fail or behave unpredictably

Key lesson:
Asking nicely for JSON is NOT enough.

This is why naive JSON generation is unsafe for:
- APIs
- Automation
- Agents


#### **💥 Topic 3 — Common JSON Failure Modes in LLM Outputs**

    Even when an LLM *tries* to return JSON, it often fails in predictable ways.
    A senior GenAI engineer must recognize these patterns immediately.

This section catalogs the **most common failure modes** seen in production.

#### ❌ Failure Mode 1 — Extra Text Before or After JSON

#### Example:
Sure! Here is the JSON you requested:

```json
{
  "name": "Dhiru",
  "age": 36
}

Why this fails:
    
- JSON parsers expect **pure JSON**
- Extra text causes parsing errors

Root cause:
LLMs are optimized to be polite and explanatory.

#### **❌ Failure Mode 2 — Markdown Wrapping**

#### **Example:**
```json
{
  "name": "Dhiru",
  "age": 36
}

#### **Why this fails:**

- Markdown is NOT valid JSON
- Parsers cannot handle ```json fences

#### **LLMs assume output is for humans, not machines.**

In [50]:
---

```markdown
## ❌ Failure Mode 3 — Missing or Extra Fields

### Example:
{
  "name": "Dhiru"
}

(Expected "age", but missing)

OR

{
  "name": "Dhiru",
  "age": 36,
  "hobby": "unknown"
}

(Unexpected field added)

### Why this fails:
- Downstream systems expect fixed schemas
- Missing or extra keys break contracts

### Root cause:
LLMs try to be helpful by guessing.

SyntaxError: invalid syntax (1202517609.py, line 1)

#### **❌ Failure Mode 4 — Wrong Data Types**

#### **Example:**

```json
{
  "name": "Dhiru",
  "age": "thirty-six"
}

#### Why this fails:
- JSON schema expects numbers, not strings
- Type mismatches cause runtime errors

#### Root cause:
LLMs reason in language, not schemas.


#### **❌ Failure Mode 5 — Invalid JSON Syntax**

#### **Example:**

```json
{
  "name": "Dhiru",
  "age": "36"
}

### Why this fails:
- Keys must be quoted
- Trailing commas are invalid

### Root cause:
LLMs approximate syntax unless constrained.


#### **🧠 Senior Engineer Insight**

These failures are NOT rare edge cases.
They are **guaranteed** to occur without enforcement.

That is why:
- Prompt discipline
- Validation
- Guardrails

are mandatory in production GenAI systems.


#### **Mental Checklist (Interview-Ready)**

When you see LLM-generated JSON, ask:
- Is it pure JSON?
- Are all required fields present?
- Are data types correct?
- Is there any extra text?
- Can a parser consume this safely?

If any answer is “no” → the system is unsafe.


#### **✅ Topic 3 Final Summary (Rule Applied)**

Identified all common JSON failure modes

Learned why they occur

Built pattern recognition for production bugs

Gained interview-level judgment

#### **Topic 4 - Enforcing JSON Output (Core Technique)**

#### **🎯 Topic 4 Goal — Enforcing JSON Output**

The goal is NOT to “ask nicely” for JSON.

The goal is to:
- Remove ambiguity
- Remove freedom to add explanations
- Force the model to behave like a data generator

We do this using:
- Clear system instructions
- Explicit output rules
- No conversational language


#### **Enforced JSON Prompt (Correct Approach)**

In [ ]:
# ============================================================
# 📘 SECTION 1 — Define a STRICT JSON Schema in the Prompt
# ------------------------------------------------------------
# Why?
#   - LLMs do not know schemas unless we tell them
#   - Explicit rules reduce hallucination
#   - We remove conversational freedom
# ============================================================

prompt = """

You must return ONLY valid JSON.
Do not include explanations, markdown, or extra text.

The JSON schema MUST be exactly:

{
    'name': string,
    'age': number,
    "skills": array of strings
}

Populate the JSON using the following data:
Name: Dhiru
Age:  36
Skill: Python, SQL, GenAI

"""

# ============================================================
# 📘 SECTION 2 — Call the LLM with Low Creativity
# ------------------------------------------------------------
# Why low temperature?
#   - Creativity increases risk of extra text
#   - Structured output requires precision
# ============================================================

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {
            "role":"system",
            "content":"You are a strict JSON generator. Output only valid JSON"
        },

        {
            "role":"user",
            "content": prompt
        }
    ],

    temperature=0.1, # low creativity
    top_p=0.9,
    max_tokens=150
)

# ============================================================
# 📘 SECTION 3 — Extract Raw Output
# ------------------------------------------------------------
# Why inspect raw output?
#   - We must verify that no extra text exists
#   - This is what downstream parsers will receive
# ============================================================

print("===== ENFORCED JSON OUTPUT =====\n")
print(raw_output)

===== ENFORCED JSON OUTPUT =====

{"name": "Dhiru", "age": 36, "skills": ["Python", "SQL", "GenAI"]}


#### **🔐 Why Enforcing JSON Works**

- The system role removes conversational behavior
- Explicit schema reduces hallucination
- Low temperature reduces creativity
- Clear rules remove ambiguity

Key idea:
LLMs follow instructions probabilistically.
Clear constraints → predictable output.


#### **✅ Topic 4 Final Summary (Rule Applied)**

- Used strict system + user instructions
- Removed conversational freedom
- Produced parser-safe JSON
- Learned how to control model behavior

#### **TOPIC 5 - Validation & Safety for Structured JSON Outputs**

#### **🎯 Topic 5 Goal — Validation & Safety**

Even with strict prompting, LLMs can still:
- produce invalid JSON
- miss fields
- return wrong data types

In production systems:

❌ Trusting the model blindly is dangerous  
✅ Validation is non-negotiable

This topic explains:
- Why validation is required
- What can go wrong
- How professionals design safety layers


#### **🧠 Core Principle (Lock This In)**

#### 🔐 Core Safety Rule

> LLM output is **untrusted input**.

Treat LLM responses exactly like:
- user input
- external API data
- uploaded files

Everything must be validated before use.



#### **Basic JSON Validation (Minimal & Clear)**

In [ ]:
# ============================================================
# 📘 SECTION 1 — Import json Library
# ------------------------------------------------------------
# Why?
#   - To parse and validate JSON strings
#   - json.loads() will fail if output is invalid
# ============================================================

import json

# ============================================================
# 📘 SECTION 2 — Raw Output from the LLM
# ------------------------------------------------------------
# Why?
#   - This simulates output coming from the model
#   - In real systems, this comes from response.content
# ============================================================

raw_output = """
{
    "name" = "Dhiru",
    "age"  =  36,
    "skills" = ["Python", "SQL", "GenAI"]
}

"""
# ============================================================
# 📘 SECTION 3 — Validate JSON Format
# ------------------------------------------------------------
# Why try/except?
#   - Invalid JSON will raise an exception
#   - We must catch failures safely
# ============================================================

try:
    parsed_output = json.loads(raw_output)
    print("✅ JSON is valid.")
    print(parsed_output)

except json.JSONDecodeError as e:
    print("❌ Invalid JSON detected.")
    print("Error:", e)

❌ Invalid JSON detected.
Error: Expecting ':' delimiter: line 3 column 12 (char 14)


#### **🔍 Validation Is More Than Syntax**

Even valid JSON can be unsafe.

You must also check:
- Required fields exist
- No unexpected fields
- Correct data types
- Reasonable values

Example:

"age": -100  ❌
"skills": "Python" ❌ (should be list)

These checks protect production systems.


#### **🧠 Production Safety Layers**

A professional GenAI system uses:

1️⃣ Prompt constraints  
2️⃣ JSON parsing  
3️⃣ Schema validation  
4️⃣ Business rule validation  
5️⃣ Fallback / retry logic  

Never rely on just one layer.


#### **❌ “The model usually returns valid JSON.”**

This mindset causes:
- Silent failures
- Security issues
- Broken automation

Production systems assume failure by default.


#### **✅ Topic 5 Final Summary (Rule Applied)**

- LLM output must be treated as untrusted input
- JSON syntax validation is mandatory
- Semantic validation is equally important
- Safety is a design responsibility, not a model feature

#### **🧭 Topic 6 — When NOT to Use JSON TOPIC 6 (Design Judgment)**

Structured JSON output is powerful — but **not always the right choice**.

Senior GenAI engineers know **when to avoid JSON** to keep systems:
- simpler
- faster
- more maintainable

This section explains those scenarios.


#### **❌ Scenario 1 — Conversational Chat Responses**

#### Example:

- Chatbots
- Tutors
- Assistants
- Q&A systems

Why JSON is a bad fit:
- Humans expect natural language
- JSON adds unnecessary complexity
- Parsing adds no value

✅ Use free-text instead.


#### **❌ Scenario 1 — Conversational Chat Responses**

#### Example:
- Chatbots
- Tutors
- Assistants
- Q&A systems

Why JSON is a bad fit:
- Humans expect natural language
- JSON adds unnecessary complexity
- Parsing adds no value

✅ Use free-text instead.

#### **❌ Scenario 2 — Creative or Open-Ended Tasks**

#### Examples:
- Story writing
- Brainstorming
- Explanations
- Analogies

Why JSON fails here:
- Creativity doesn’t fit rigid schemas
- Over-constraining reduces quality
- Output feels unnatural

✅ Use free-text with temperature control.


#### **❌ Scenario 3 — Simple One-Off Answers**

#### **Example:**
“What is Python?”

Why JSON is unnecessary:
- No downstream parsing
- No automation
- No structured consumption

JSON here is **overengineering**.


#### **⚠️ Scenario 4 — Partial or Hybrid Needs**

Sometimes you need:
- One structured field
- Plus human explanation

Example:

  ```json
{
 "decision": "approve",
 "reason": "..."
}

In these cases:
- Use hybrid outputs
- Or separate calls:
  - One JSON call
  - One text call


#### **🧠 Senior Engineer Rule of Thumb**

Use JSON when:
- Output feeds another system
- Automation depends on it
- Tools or agents consume it

Avoid JSON when:
- Output is read by humans
- Creativity matters
- No parsing is required

#### **Interview-Level Insight**

Question:
"When would you avoid structured outputs?"

Strong answer:
"When the output is purely conversational or creative and no downstream system needs parsing."

Weak answer:
"JSON is always better."


#### **✅ Topic 6 Summary**

- JSON is not a default choice
- Overusing JSON increases complexity
- Senior engineers choose structure intentionally
- Design judgment matters more than syntax


### **TOPIC 7 - Micro Practice: Break & Fix JSON (Hands-on)**

The goal of this practice is:
- NOT to memorize syntax
- BUT to build confidence handling failures

We will:
1) Intentionally break JSON
2) Observe how validation fails
3) Fix it using better prompting and validation

This mirrors real production debugging.


#### **Intentionally Broken JSON Output**

In [ ]:
# ============================================================
# 📘 STEP 1 — Simulate a Broken JSON Output
# ------------------------------------------------------------
# Why simulate?
#   - In real systems, LLMs sometimes return invalid JSON
#   - We must be prepared to handle it safely
# ============================================================

broken_output ="""

Sure! Here is the JSON you requested:

{
    name: "Dhiru",
    age: "thirty-six",
    skills: ['Python', 'SQL', "GenAI"]
}

"""

print("===== BROKEN OUTPUT =====\n")
print(broken_output)

===== BROKEN OUTPUT =====



Sure! Here is the JSON you requested:

{
    name: "Dhiru",
    age: "thirty-six",
    skills: ['Python', 'SQL', "GenAI"]
}




#### **Validate and Observe Failure**

In [ ]:
# ============================================================
# 📘 STEP 2 — Attempt to Parse Broken JSON
# ------------------------------------------------------------
# Why?
#   - To prove that parsers fail on invalid JSON
#   - This is our safety checkpoint
# ============================================================

"""import json

try:
    parsed = json.load(broken_output)
    print("✅ JSON parsed successfully:", parsed)

except json.JSONDecodeError as e:
    print("❌ JSON parsing failed.")   print("Error:", e)"""

'import json\n\ntry:\n    parsed = json.load(broken_output)\n    print("✅ JSON parsed successfully:", parsed)\n\nexcept json.JSONDecodeError as e:\n    print("❌ JSON parsing failed.")   print("Error:", e)'

#### **Fix the JSON (Enforced Output)**

In [55]:
# ============================================================
# 📘 STEP 3 — Correct JSON Output
# ------------------------------------------------------------
# Why?
#   - Remove extra text
#   - Fix data types
#   - Ensure valid syntax
# ============================================================

fixed_output = """

{
    "name" = "Dhiru",
    "age" = 36,
    "skills" = ["Python", "SQL", "GenAI"]
}

"""
try:
    parsed_fixed = json.loads(fixed_output)
    print("✅ Fixed JSON is valid.")
    print(parsed_fixed)

except json.JSONDecodeError as e:
    print("❌ Still invalid JSON:", e)

❌ Still invalid JSON: Expecting ':' delimiter: line 4 column 12 (char 15)



- Even small formatting issues break JSON parsing
- Extra text and wrong data types are common failures
- Validation catches issues early
- Fixing JSON is easier than debugging broken pipelines

Key lesson:
- Never trust LLM output without validation.

#### **✅ Topic 7 Final Summary (Rule Applied)**

Practiced breaking JSON intentionally

Observed parser failures

Fixed structure and data types

Built real-world debugging confidence

#### **TOPIC 8 - Mini Mock Test (Structured Outputs & JSON)**
#### **🧪 Mini Mock Test — Structured Outputs (JSON)**

##### **Answer these questions in your own words.**
---

**Q1️⃣ Why is naive JSON generation unsafe in production systems?**

    A1: Because LLMs Often add extra text, invalid syntax, or hallucinated fields, which breaks parsers.

---

**Q2️⃣ List any **three common JSON failure modes** you observed when using LLMs.**

    A2: Extra text, markdown wrapping, missing fields, wrong data types, invalid syntax.
---

**Q3️⃣ Why does adding “Here is the JSON” break downstream systems?**

    A3: Parsers expect pure JSON, any extra text causes parsing errors.
---

**Q4️⃣ What is the role of the **system message** when enforcing JSON output?**

    A4: The system message removes conversational behavior and enforces strict output rules.
---

**Q5️⃣ True or False:**
“If the JSON looks correct to humans, it is safe for machines.”
    
    A5: False.
---

**Q6️⃣ Why is JSON validation mandatory even after strict prompting?**

    A6: Because even valid-looking JSON can have missing fields, wrong types, or unsafe values.
---

**Q7️⃣ In which scenarios should JSON **NOT** be used, and why?**

    A7: Conversational Chat, creative tasks, simple explanations-because parsing is unnecessary and JSON adds complexity.
---
**Q8️⃣ How would you explain structured outputs to an interviewer in one sentence?**

    A8: "Structured outputs ensure LLM responses are deterministic, parseable, and safe for production systems."